In [ ]:
from parser import Parser
from summarizer import Summarizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
paper_url = "https://arxiv.org/pdf/2305.14314"
paper = Parser.parse(paper_url)

model = Summarizer(device=device)
result = model.summarize(paper)

In [ ]:
import os

with open("summary.md", "w") as f:
    f.write(result)